# Module 3: Land Classification — CNN–Transformer Integration Evaluation

In [ ]:
import os, glob, numpy as np, matplotlib.pyplot as plt
from PIL import Image, ImageDraw

DATASET_DIR = "./images_dataSAT"
DIR_NON_AGRI = os.path.join(DATASET_DIR, "class_0_non_agri")
DIR_AGRI = os.path.join(DATASET_DIR, "class_1_agri")

def _ensure_dataset():
    os.makedirs(DIR_NON_AGRI, exist_ok=True)
    os.makedirs(DIR_AGRI, exist_ok=True)
    if len(os.listdir(DIR_NON_AGRI))>0 and len(os.listdir(DIR_AGRI))>0:
        return
    import numpy as np
    from PIL import Image, ImageDraw
    rng = np.random.default_rng(0)
    for cls_dir, pattern in [(DIR_NON_AGRI, 'rect'), (DIR_AGRI, 'lines')]:
        for i in range(12):
            img = Image.new("RGB",(64,64),(rng.integers(20,235),rng.integers(20,235),rng.integers(20,235)))
            d = ImageDraw.Draw(img)
            if pattern=='rect':
                d.rectangle([10,10,54,54], outline=(255,255,255), width=2)
            else:
                for y in range(5,64,10):
                    d.line([0,y,64,y], fill=(255,255,255), width=1)
            img.save(os.path.join(cls_dir, f"img_{{i:03d}}.png"))

# Copy dataset from /mnt/data if available
if os.path.exists('/mnt/data/images_dataSAT'):
    import shutil
    if not os.path.exists(DATASET_DIR):
        shutil.copytree('/mnt/data/images_dataSAT', DATASET_DIR)
_ensure_dataset()
print("Dataset ready at", os.path.abspath(DATASET_DIR))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

DATASET_DIR = "./images_dataSAT"
HYPERPARAMS = {"image_size": (64,64), "batch_size": 16, "epochs": 2}
print("Dataset:", DATASET_DIR); print("Hyperparams:", HYPERPARAMS)
def print_metrics(y_true, y_prob):
    y_pred = (y_prob > 0.5).astype(int)
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, zero_division=0))
    print("Recall:", recall_score(y_true, y_pred, zero_division=0))
    print("F1:", f1_score(y_true, y_pred, zero_division=0))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

In [ ]:
# Keras hybrid evaluation
import tensorflow as tf
from tensorflow.keras import layers, models

def build_keras_hybrid():
    inp = layers.Input((64,64,3))
    x = layers.Rescaling(1./255)(inp)
    x = layers.Conv2D(16,3,activation='relu')(x); x = layers.MaxPooling2D()(x)
    patches = tf.image.extract_patches(x, sizes=[1,8,8,1], strides=[1,8,8,1], rates=[1,1,1,1], padding='VALID')
    patches = layers.Reshape((-1, patches.shape[-1]))(patches)
    z = layers.MultiHeadAttention(num_heads=2, key_dim=32)(patches, patches)
    z = layers.GlobalAveragePooling1D()(z)
    out = layers.Dense(1, activation='sigmoid')(z)
    return models.Model(inp, out)

ds_train = tf.keras.utils.image_dataset_from_directory(DATASET_DIR, validation_split=0.3, subset='training', seed=123, image_size=(64,64), batch_size=16)
ds_val = tf.keras.utils.image_dataset_from_directory(DATASET_DIR, validation_split=0.3, subset='validation', seed=123, image_size=(64,64), batch_size=16)
model_k = build_keras_hybrid(); model_k.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_k.fit(ds_train, validation_data=ds_val, epochs=2, verbose=0)
y_true, y_prob = [], []
for X,y in ds_val:
    p = model_k.predict(X, verbose=0).ravel()
    y_true.extend(y.numpy().ravel()); y_prob.extend(p)
print("Keras hybrid metrics:"); print_metrics(np.array(y_true), np.array(y_prob))

In [ ]:
# PyTorch hybrid evaluation
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
full = datasets.ImageFolder(DATASET_DIR, transform=transform)
n_val = int(0.3*len(full)); n_train = len(full)-n_val
train_subset, val_subset = torch.utils.data.random_split(full, [n_train, n_val], generator=torch.Generator().manual_seed(123))
train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=16, shuffle=False)

class CNNViTPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Sequential(nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2))
        self.proj = nn.Conv2d(16,32,kernel_size=8, stride=8)
        encoder_layer = nn.TransformerEncoderLayer(d_model=32, nhead=2, batch_first=True)
        self.enc = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.fc = nn.Linear(32,1)
    def forward(self,x):
        x = self.stem(x); x = self.proj(x)
        B,C,H,W = x.shape
        x = x.flatten(2).transpose(1,2)
        x = self.enc(x).mean(dim=1)
        return self.fc(x).squeeze(1)

device = "cuda" if torch.cuda.is_available() else "cpu"
m = CNNViTPT().to(device); opt = torch.optim.Adam(m.parameters(), 1e-3); crit = nn.BCEWithLogitsLoss()
for _ in range(2):
    m.train()
    for X,y in train_loader:
        X, y = X.to(device), y.float().to(device)
        opt.zero_grad(); out = m(X); loss = crit(out,y); loss.backward(); opt.step()

m.eval(); probs=[]; trues=[]
with torch.no_grad():
    for X,y in val_loader:
        X = X.to(device); out = m(X)
        probs.extend(torch.sigmoid(out).cpu().numpy().ravel()); trues.extend(y.numpy().ravel())
print("PyTorch hybrid metrics:"); print_metrics(np.array(trues), np.array(probs))